In [1]:
import ee
import geopandas as gpd
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Optional
import urllib.request
from calendar import monthrange
import requests
from requests.exceptions import Timeout as RequestsTimeout, RequestException, HTTPError
from geobr import read_municipality
import math
import time
import json
import socket


In [2]:
ee.Initialize(project="cropyieldprediction-476612")

In [3]:
class GEEDownloader:
    """Downloader for Sentinel-2 imagery with MapBiomas crop masking from Google Earth Engine."""
    
    def __init__(self, output_dir: str = "files/gee_images_30m", failed_municipalities_file: str = "files/failed_municipalities_30m.json"):
        """
        Initialize the GEE downloader.
        
        Parameters:
        -----------
        output_dir : str, default "files/gee_images"
            Directory where downloaded images will be saved.
        failed_municipalities_file : str, default "files/failed_municipalities.json"
            Path to JSON file storing list of municipalities that failed to download.
        """
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.failed_municipalities_file = Path(failed_municipalities_file)
        self.failed_municipalities_file.parent.mkdir(parents=True, exist_ok=True)
        self.mapbiomas_asset = "projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_integration_v2"
        self.sentinel2_collection = "COPERNICUS/S2_SR_HARMONIZED"
    
    def download_municipality(
        self,
        shapefile_path: str,
        crop_type: int = 39,
        resolution: int = 30,
        tile_size: int = 15000,
        start_date: str = "2023-01-01",
        end_date: str = "2023-12-31",
        cloud_threshold: float = 30.0,
        composite_method: str = "median",
        bands: List[str] = ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"],
        municipality_code: Optional[str] = None,
        municipality_name: Optional[str] = None,
        force_redownload: bool = False,
        timeout_minutes: int = 60
    ) -> Path:
        """
        Download Sentinel-2 imagery for a municipality with MapBiomas crop masking.
        
        Downloads monthly composite images divided into fixed-size tiles. Only tiles
        containing the specified crop type are downloaded. Images are masked to show
        only pixels classified as the target crop in MapBiomas.
        
        Parameters:
        -----------
        shapefile_path : str
            Path to the municipality shapefile (.shp file).
        crop_type : int, default 39
            MapBiomas classification code for the crop type (39 = soybean).
        resolution : int, default 30
            Pixel resolution in meters for the downloaded images.
        tile_size : int, default 1000
            Size of each tile in meters (e.g., 1000 = 1km x 1km tiles).
            Note: In practice, larger values (e.g., 5000) are often used for efficiency.
        start_date : str, default "2023-01-01"
            Start date for the download period (YYYY-MM-DD format).
        end_date : str, default "2023-12-31"
            End date for the download period (YYYY-MM-DD format).
        cloud_threshold : float, default 30.0
            Maximum cloud cover percentage for images to be included in composite.
        composite_method : str, default "median"
            Method to composite multiple images per month: "median", "mean", or "first".
        bands : List[str], default ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"]
            List of Sentinel-2 bands to download (excluding B1, B9, B10).
        municipality_code : Optional[str], default None
            IBGE municipality code. If None, extracted from shapefile filename.
        municipality_name : Optional[str], default None
            Municipality name. If None, uses municipality_code.
        force_redownload : bool, default False
            If True, re-downloads even if files already exist. If False, skips
            municipalities that have already been downloaded for the requested date range.
        
        Returns:
        --------
        Path
            Path to the directory containing downloaded GeoTIFF files.
            Files are named: {municipality_code}_{year}-{month}_tile_{i:02d}_{j:02d}.tif
        """
        
        gdf = gpd.read_file(shapefile_path)
        geometry = gdf.geometry.values[0]
        geo_json = geometry.__geo_interface__
        ee_geometry = ee.Geometry(geo_json)
        
        if not municipality_code:
            municipality_code = Path(shapefile_path).stem
        if not municipality_name:
            municipality_name = municipality_code
        
        safe_name = "".join(c for c in municipality_name if c.isalnum() or c in (' ', '-', '_')).strip()
        mun_output_dir = self.output_dir / f"{municipality_code}_{safe_name}"
        
        if not force_redownload and mun_output_dir.exists():
            existing_files = list(mun_output_dir.glob("*.tif"))
            if existing_files:
                print(f"Skipping {municipality_code} ({municipality_name}): folder exists with {len(existing_files)} files")
                return mun_output_dir
        
        municipality_start_time = time.time()
        print(f"\n[{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}] Starting processing: {municipality_code} ({municipality_name})")
        
        mun_output_dir.mkdir(parents=True, exist_ok=True)
        
        TIMEOUT_SECONDS = 60 * timeout_minutes
        
        gdf_bounds = gdf.total_bounds
        min_lon, min_lat = gdf_bounds[0], gdf_bounds[1]
        max_lon, max_lat = gdf_bounds[2], gdf_bounds[3]
        
        center_lat = (min_lat + max_lat) / 2
        
        meters_per_degree_lat = 111000
        meters_per_degree_lon = 111000 * math.cos(math.radians(center_lat))
        
        tile_size_deg_lat = tile_size / meters_per_degree_lat
        tile_size_deg_lon = tile_size / meters_per_degree_lon
        
        num_tiles_lon = int((max_lon - min_lon) / tile_size_deg_lon) + 1
        num_tiles_lat = int((max_lat - min_lat) / tile_size_deg_lat) + 1
        
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        
        current = start
        while current <= end:
            year = current.year
            month = current.month
            month_end = datetime(year, month, monthrange(year, month)[1])
            if month_end > end:
                month_end = end
            
            month_start_str = current.strftime("%Y-%m-%d")
            month_end_str = month_end.strftime("%Y-%m-%d")
            
            print(f"[LOG] Processing {year}-{month:02d}...")
            
            sentinel2 = (
                ee.ImageCollection(self.sentinel2_collection)
                .filterBounds(ee_geometry)
                .filterDate(month_start_str, month_end_str)
                .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud_threshold))
            )
            
            try:
                if composite_method == "median":
                    composite = sentinel2.median()
                elif composite_method == "mean":
                    composite = sentinel2.mean()
                elif composite_method == "first":
                    composite = sentinel2.first()
                else:
                    composite = sentinel2.median()
                
                rgb = composite.select(bands).clip(ee_geometry)
                
                mapbiomas = ee.Image(self.mapbiomas_asset)
                classification = mapbiomas.select(f"classification_{year}").clip(ee_geometry)
                crop_mask = classification.eq(crop_type)
                
                rgb_masked = rgb.multiply(crop_mask).updateMask(crop_mask)
            except (ee.EEException, Exception) as e:
                error_msg = str(e)
                if "Band pattern" in error_msg and "no bands" in error_msg:
                    print(f"[LOG] Skipping {year}-{month:02d} - no bands")
                    current = month_end + timedelta(days=1)
                    continue
                elif "geometry for image clipping must not be empty" in error_msg:
                    print(f"[LOG] Skipping {year}-{month:02d} - empty geometry")
                    current = month_end + timedelta(days=1)
                    continue
                else:
                    print(f"[LOG] Error creating composite for {year}-{month:02d}: {error_msg}")
                    raise
            
            tiles_downloaded_this_month = 0
            for i in range(num_tiles_lon):
                for j in range(num_tiles_lat):
                    elapsed_time = time.time() - municipality_start_time
                    
                    tile_lon_min = min_lon + (i * tile_size_deg_lon)
                    tile_lon_max = min_lon + ((i + 1) * tile_size_deg_lon)
                    tile_lat_min = min_lat + (j * tile_size_deg_lat)
                    tile_lat_max = min_lat + ((j + 1) * tile_size_deg_lat)
                    
                    tile_geometry = ee.Geometry.Rectangle(
                        [tile_lon_min, tile_lat_min, tile_lon_max, tile_lat_max]
                    )
                    tile_geometry = tile_geometry.intersection(ee_geometry, ee.ErrorMargin(1))
                    
                    elapsed_time = time.time() - municipality_start_time
                    
                    url = None  
                    try:
                        tile_rgb_masked = rgb_masked.clip(tile_geometry)
                                                
                        url = tile_rgb_masked.getDownloadUrl({
                            "region": tile_geometry,
                            "scale": resolution,
                            "crs": "EPSG:4326",
                            "format": "GEO_TIFF"
                        })
                                                
                        filename = f"{municipality_code}_{year}-{month:02d}_tile_{i:02d}_{j:02d}.tif"
                        filepath = mun_output_dir / filename
                        mun_output_dir.mkdir(parents=True, exist_ok=True)
                        
                        response = requests.get(url, timeout=TIMEOUT_SECONDS, stream=True)
                        response.raise_for_status()
                        with open(filepath, 'wb') as f:
                            for chunk in response.iter_content(chunk_size=8192):
                                elapsed_time = time.time() - municipality_start_time
                                if elapsed_time > TIMEOUT_SECONDS:
                                    raise RequestsTimeout(f"Municipality timeout exceeded during download")
                                if chunk:
                                    f.write(chunk)
                        tiles_downloaded_this_month += 1
                        
                    except HTTPError as e:
                        error_msg = str(e)
                        if e.response is not None and e.response.status_code == 503:
                            print(f"[LOG] 503 Server Error: Service Unavailable for url: {url}")
                            self._save_failed_municipality(
                                municipality_code, 
                                municipality_name, 
                                "503_service_unavailable",
                                url=url,
                                error_details=error_msg
                            )
                            continue
                        else:
                            print(f"[LOG] HTTP Error {e.response.status_code if e.response else 'unknown'}: {error_msg}")
                            if url:
                                self._save_failed_municipality(
                                    municipality_code,
                                    municipality_name,
                                    f"http_error_{e.response.status_code if e.response else 'unknown'}",
                                    url=url,
                                    error_details=error_msg
                                )
                            continue
                    except RequestsTimeout as e:
                        print(f"[LOG] TIMEOUT: Municipality timeout exceeded ({elapsed_time/60:.1f} minutes) for {municipality_code}.")
                        self._save_failed_municipality(municipality_code, municipality_name, "timeout")
                        return mun_output_dir
                    except (ee.EEException, RequestException, Exception) as e:
                        error_msg = str(e)
                        if "geometry for image clipping must not be empty" in error_msg:
                            continue
                        elif "Total request size" in error_msg and "must be less than" in error_msg:
                            continue
                        if "503" in error_msg and "Service Unavailable" in error_msg:
                            url_match = None
                            if "for url:" in error_msg:
                                url_match = error_msg.split("for url:")[-1].strip()
                            print(f"[LOG] 503 Server Error detected: {error_msg}")
                            self._save_failed_municipality(
                                municipality_code,
                                municipality_name,
                                "503_service_unavailable",
                                url=url_match if url_match else url,
                                error_details=error_msg
                            )
                            continue
                        print(error_msg)
                        continue
            
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{timestamp}] Completed {year}-{month:02d}: {tiles_downloaded_this_month} tiles")
            
            current = month_end + timedelta(days=1)
        
        elapsed_time = time.time() - municipality_start_time
        elapsed_minutes = int(elapsed_time // 60)
        elapsed_seconds = int(elapsed_time % 60)
        end_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        num_files = len(list(mun_output_dir.glob("*.tif")))
        print(f"[{end_timestamp}] Completed: {municipality_code} ({municipality_name}) - {num_files} files in {elapsed_minutes}m {elapsed_seconds}s")
        
        return mun_output_dir
    
    def _save_failed_municipality(self, municipality_code: str, municipality_name: str, reason: str, url: Optional[str] = None, error_details: Optional[str] = None):
        """
        Save a failed municipality to the failed municipalities list.
        
        Parameters:
        -----------
        municipality_code : str
            Municipality code.
        municipality_name : str
            Municipality name.
        reason : str
            Reason for failure (e.g., "timeout", "503_service_unavailable").
        url : Optional[str], default None
            URL that failed (e.g., the GEE download URL).
        error_details : Optional[str], default None
            Additional error details or full error message.
        """
        if self.failed_municipalities_file.exists():
            with open(self.failed_municipalities_file, 'r') as f:
                failed_list = json.load(f)
        else:
            failed_list = []
        
        failed_entry = {
            "code": municipality_code,
            "name": municipality_name,
            "reason": reason,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        # Add URL if provided
        if url:
            failed_entry["url"] = url
        
        # Add error details if provided
        if error_details:
            failed_entry["error_details"] = error_details
        
        # Check if this municipality already exists in the list
        existing_entry = None
        for entry in failed_list:
            if entry["code"] == municipality_code:
                existing_entry = entry
                break
        
        if existing_entry:
            # Update existing entry - add new URL if not already present
            if url and "urls" not in existing_entry:
                # Convert single url to urls list for backward compatibility
                if "url" in existing_entry:
                    existing_entry["urls"] = [existing_entry.pop("url")]
                else:
                    existing_entry["urls"] = []
            
            if url:
                if "urls" in existing_entry:
                    if url not in existing_entry["urls"]:
                        existing_entry["urls"].append(url)
                elif "url" in existing_entry:
                    if existing_entry["url"] != url:
                        existing_entry["urls"] = [existing_entry.pop("url"), url]
                else:
                    existing_entry["url"] = url
            
            # Update error details
            if error_details:
                if "error_details" not in existing_entry:
                    existing_entry["error_details"] = []
                elif isinstance(existing_entry["error_details"], str):
                    existing_entry["error_details"] = [existing_entry["error_details"]]
                
                if isinstance(existing_entry["error_details"], list):
                    if error_details not in existing_entry["error_details"]:
                        existing_entry["error_details"].append(error_details)
                else:
                    existing_entry["error_details"] = [str(existing_entry["error_details"]), error_details]
            
            # Update timestamp
            existing_entry["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        else:
            # Add new entry
            failed_list.append(failed_entry)
        
        with open(self.failed_municipalities_file, 'w') as f:
            json.dump(failed_list, f, indent=2)
    
    def download_specific_url(self, url: str, municipality_code: str, municipality_name: str, output_filename: str, timeout_minutes: int = 60):
        """
        Download a specific URL (e.g., for retrying failed 503 errors).
        
        Parameters:
        -----------
        url : str
            The GEE download URL to retry.
        municipality_code : str
            Municipality code.
        municipality_name : str
            Municipality name.
        output_filename : str
            Filename to save the downloaded file (e.g., "4104402_2023-04_tile_00_05.tif").
        timeout_minutes : int, default 60
            Timeout in minutes for the download.
        """
        safe_name = "".join(c for c in municipality_name if c.isalnum() or c in (' ', '-', '_')).strip()
        mun_output_dir = self.output_dir / f"{municipality_code}_{safe_name}"
        mun_output_dir.mkdir(parents=True, exist_ok=True)
        
        filepath = mun_output_dir / output_filename
        TIMEOUT_SECONDS = 60 * timeout_minutes
        
        print(f"[LOG] Retrying download for {municipality_code} ({municipality_name}): {output_filename}")
        try:
            response = requests.get(url, timeout=TIMEOUT_SECONDS, stream=True)
            response.raise_for_status()
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            print(f"[LOG] Successfully downloaded: {filepath}")
            return filepath
        except Exception as e:
            print(f"[LOG] Failed to download {url}: {str(e)}")
            raise


In [4]:
shapefile_dir = Path("files/municipal_shapefiles")
shapefiles = list(shapefile_dir.glob("**/*.shp"))

for shp_file in shapefiles:
    print(f"Using shapefile: {shp_file}")
    
    gdf = gpd.read_file(shp_file)
    mun_code = shp_file.stem
    mun_name = shp_file.parent.name.split('_', 1)[1] if '_' in shp_file.parent.name else mun_code
    
    downloader = GEEDownloader()
    output_path = downloader.download_municipality(
        shapefile_path=str(shp_file),
        crop_type=39,
        resolution=30,
        tile_size=5000,
        start_date="2023-01-01",
        end_date="2023-06-30",
        cloud_threshold=30.0,
        composite_method="median",
        bands=["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"],
        municipality_code=mun_code,
        municipality_name=mun_name,
        timeout_minutes=20
    )
    
    print(f"Download complete! Files saved to: {output_path}")
else:
    print("No shapefiles found in files/municipal_shapefiles/")


Using shapefile: files\municipal_shapefiles\4100103_Abatiá\4100103.shp
Skipping 4100103 (Abatiá): folder exists with 95 files
Download complete! Files saved to: files\gee_images_30m\4100103_Abatiá
Using shapefile: files\municipal_shapefiles\4100202_Adrianópolis\4100202.shp
Skipping 4100202 (Adrianópolis): folder exists with 462 files
Download complete! Files saved to: files\gee_images_30m\4100202_Adrianópolis
Using shapefile: files\municipal_shapefiles\4100301_Agudos do Sul\4100301.shp
Skipping 4100301 (Agudos do Sul): folder exists with 78 files
Download complete! Files saved to: files\gee_images_30m\4100301_Agudos do Sul
Using shapefile: files\municipal_shapefiles\4100400_Almirante Tamandaré\4100400.shp
Skipping 4100400 (Almirante Tamandaré): folder exists with 108 files
Download complete! Files saved to: files\gee_images_30m\4100400_Almirante Tamandaré
Using shapefile: files\municipal_shapefiles\4100459_Altamira do Paraná\4100459.shp
Skipping 4100459 (Altamira do Paraná): folder exi

KeyboardInterrupt: 

In [ ]:
import shutil

# Read failed municipalities
failed_file = Path("files/failed_municipalities_30m.json")
if failed_file.exists():
    with open(failed_file, 'r') as f:
        failed_municipalities = json.load(f)
    
    downloader = GEEDownloader()
    successfully_retried = []  # Track successfully retried municipalities
    
    for failed_entry in failed_municipalities:
        mun_code = failed_entry["code"]
        mun_name = failed_entry["name"]
        reason = failed_entry["reason"]
        
        print(f"\n{'='*60}")
        print(f"Processing failed municipality: {mun_code} ({mun_name})")
        print(f"Reason: {reason}")
        print(f"{'='*60}\n")
        
        if reason == "timeout":
            # Delete the folder and restart download
            safe_name = "".join(c for c in mun_name if c.isalnum() or c in (' ', '-', '_')).strip()
            mun_output_dir = downloader.output_dir / f"{mun_code}_{safe_name}"
            
            if mun_output_dir.exists():
                print(f"[LOG] Deleting existing folder: {mun_output_dir}")
                shutil.rmtree(mun_output_dir)
            
            # Find the shapefile
            shapefile_path = Path(f"files/municipal_shapefiles/{mun_code}_{mun_name}/{mun_code}.shp")
            if not shapefile_path.exists():
                # Try alternative path format
                shapefile_path = Path(f"files/municipal_shapefiles") / f"{mun_code}_{mun_name}" / f"{mun_code}.shp"
            
            if shapefile_path.exists():
                print(f"[LOG] Restarting download for {mun_code} ({mun_name})")
                try:
                    output_path = downloader.download_municipality(
                        shapefile_path=str(shapefile_path),
                        crop_type=39,
                        resolution=30,
                        tile_size=5000,
                        start_date="2023-01-01",
                        end_date="2023-06-30",
                        cloud_threshold=30.0,
                        composite_method="median",
                        bands=["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"],
                        municipality_code=mun_code,
                        municipality_name=mun_name,
                        timeout_minutes=20,
                        force_redownload=True
                    )
                    print(f"[LOG] Successfully completed: {output_path}")
                    successfully_retried.append(mun_code)
                except Exception as e:
                    print(f"[LOG] Error retrying {mun_code}: {str(e)}")
            else:
                print(f"[LOG] ERROR: Shapefile not found: {shapefile_path}")
        
        elif reason == "503_service_unavailable":
            # Try to download the failed URL
            url = failed_entry.get("url")
            if url:
                safe_name = "".join(c for c in mun_name if c.isalnum() or c in (' ', '-', '_')).strip()
                mun_output_dir = downloader.output_dir / f"{mun_code}_{safe_name}"
                
                # First, try to download the URL directly
                # Extract date/month info from error_details if available, otherwise use timestamp
                error_details = failed_entry.get("error_details", "")
                filename = f"{mun_code}_retry_{datetime.now().strftime('%Y%m%d_%H%M%S')}.tif"
                
                # Try to infer filename from existing files pattern
                if mun_output_dir.exists():
                    existing_files = sorted(mun_output_dir.glob(f"{mun_code}_*.tif"))
                    if existing_files:
                        # Get the pattern from existing files
                        # Files are named like: {mun_code}_{year}-{month}_tile_{i}_{j}.tif
                        # We'll try to find which month/tile might be missing
                        # For now, use a generic retry filename
                        print(f"[LOG] Found {len(existing_files)} existing files.")
                
                print(f"[LOG] Attempting to download failed URL directly: {url[:80]}...")
                url_success = False
                try:
                    downloader.download_specific_url(
                        url=url,
                        municipality_code=mun_code,
                        municipality_name=mun_name,
                        output_filename=filename,
                        timeout_minutes=20
                    )
                    url_success = True
                    print(f"[LOG] Successfully downloaded URL to: {filename}")
                    successfully_retried.append(mun_code)
                except Exception as e:
                    print(f"[LOG] Failed to download URL (may be expired): {str(e)}")
                    print(f"[LOG] Falling back to retrying whole municipality...")
                
                # If URL download failed or we want to ensure completeness, retry whole municipality
                if not url_success:
                    # Find the shapefile
                    shapefile_path = Path(f"files/municipal_shapefiles/{mun_code}_{mun_name}/{mun_code}.shp")
                    if not shapefile_path.exists():
                        shapefile_path = Path(f"files/municipal_shapefiles") / f"{mun_code}_{mun_name}" / f"{mun_code}.shp"
                    
                    if shapefile_path.exists():
                        try:
                            output_path = downloader.download_municipality(
                                shapefile_path=str(shapefile_path),
                                crop_type=39,
                                resolution=30,
                                tile_size=5000,
                                start_date="2023-01-01",
                                end_date="2023-06-30",
                                cloud_threshold=30.0,
                                composite_method="median",
                                bands=["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"],
                                municipality_code=mun_code,
                                municipality_name=mun_name,
                                timeout_minutes=20,
                                force_redownload=False  # Don't delete existing files
                            )
                            print(f"[LOG] Successfully completed: {output_path}")
                            successfully_retried.append(mun_code)
                        except Exception as e:
                            print(f"[LOG] Error retrying {mun_code}: {str(e)}")
                    else:
                        print(f"[LOG] ERROR: Shapefile not found: {shapefile_path}")
            else:
                print(f"[LOG] ERROR: No URL found in failed entry for {mun_code}")
        
        else:
            print(f"[LOG] Unknown failure reason: {reason}. Skipping {mun_code}.")
    
    # Remove successfully retried municipalities from the failed list
    if successfully_retried:
        original_count = len(failed_municipalities)
        failed_municipalities = [entry for entry in failed_municipalities if entry["code"] not in successfully_retried]
        removed_count = original_count - len(failed_municipalities)
        
        # Save updated failed municipalities list
        with open(failed_file, 'w') as f:
            json.dump(failed_municipalities, f, indent=2)
        
        print(f"\n{'='*60}")
        print(f"Successfully retried {removed_count} municipality/municipalities: {', '.join(successfully_retried)}")
        print(f"Removed from failed list. {len(failed_municipalities)} municipalities still in failed list.")
        print(f"{'='*60}\n")
    else:
        print(f"\n{'='*60}")
        print("No municipalities were successfully retried.")
        print(f"{len(failed_municipalities)} municipalities still in failed list.")
        print(f"{'='*60}\n")
else:
    print("No failed municipalities file found: files/failed_municipalities_30m.json")



Processing failed municipality: 4107603 (Faxinal)
Reason: timeout

[LOG] Deleting existing folder: files\gee_images_30m\4107603_Faxinal
[LOG] Restarting download for 4107603 (Faxinal)

[2025-11-26 11:42:47] Starting processing: 4107603 (Faxinal)
[LOG] Processing 2023-01...
[2025-11-26 11:45:18] Completed 2023-01: 45 tiles
[LOG] Processing 2023-02...
[2025-11-26 11:47:49] Completed 2023-02: 45 tiles
[LOG] Processing 2023-03...
[LOG] 503 Server Error: Service Unavailable for url: https://earthengine.googleapis.com/v1/projects/cropyieldprediction-476612/thumbnails/dd11b3150f1ba3fd0f1ff80316ad547a-32a0bed2fde422f6ad281a7104d40774:getPixels
[2025-11-26 11:50:10] Completed 2023-03: 44 tiles
[LOG] Processing 2023-04...
